
# Exposure Calculations
In this notebook, one can set a sky position to look at (boresight position) and a source (with profile and position) and study the effect of possible telescope dithering on the required exposure time.


In [1]:
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import dithering
from scipy.interpolate import interp1d


After the general packages and the special package "dithering" is imported, one start with setting the source and boresight positions. By default, "generate_source()" function generates a source like a QSO. For other types of sources, you can uncomment/comment the lines below.


### Constructing the object

One can uncomment/comment a source below to study

In [2]:
dit    = dithering.dithering()
# QSO
#source = dit.generate_source(disk_fraction=0., bulge_fraction=0.)
#source_type = 'qso'
# ELG
source = dit.generate_source(disk_fraction=1., bulge_fraction=0.)
source_type = 'elg'
# LRG
#source = dit.generate_source(disk_fraction=0., bulge_fraction=1.)
#source_type = 'lrg'
dit.set_source_position(20.*u.deg, 25.*u.deg)
dit.set_boresight_position(20.*u.deg, 24.5*u.deg)
dit.set_focal_plane_position()
dit.run_simulation(source_type, *source, report=False)
snr = dit.SNR

### Moving the telescope
This is where the <b>magic</b> happens. The fiber is positioned at the original focal plane coordinates. When one changes the boresight position, the package automatically calculates the new focal plane coordinates along with the fiber placement offset. This offset is used to move the fiber (in turn the aperture in the fiber acceptance calculations).
In the example below, the altitude boresight is changed by <b>0.05</b> degree. 

In [3]:
dit.change_alt_az_bore_position(20.*u.deg, 24.5*u.deg+1*u.arcsec)
dit.run_simulation(source_type, *source, report=False)

### Calculate SNR.vs.exposure and estimating exposure time required

The telescope has moved by 0.05 degrees. This might be an actually systematic problem. We scan the required exposure to get the same SNR as if everything was perfectly aligned 

In [4]:
print("Exposure time was set to {0} in the configuration file".format(dit.desi.observation.exposure_time))
print("Corresponding SNRs are:")
for camera in snr:
    print("--- camera {0}: {1:.3f}".format(camera, np.median(snr[camera][0])))
print("When the telescope is moved, SNR changes to:")
for camera in dit.SNR:
    print("--- camera {0}: {1:.3f}".format(camera, np.median(dit.SNR[camera][0])))
    
exposure_time = 100.*u.s
exposure_times = np.logspace(np.log10(exposure_time.value), np.log10(exposure_time.value)+3, 20)
x = []
y_b = []
y_r = []
y_z = []
interpolator = {}
for i in range(20):
    dit.desi.observation.exposure_time = exposure_times[i]*u.s
    dit.run_simulation(source_type, *source, report=False)
    x.append(exposure_times[i])
    y_b.append(np.median(dit.SNR['b'][0]))
    y_r.append(np.median(dit.SNR['r'][0]))
    y_z.append(np.median(dit.SNR['z'][0]))
interpolator['b'] = interp1d(y_b, x, fill_value='extrapolate')
interpolator['r'] = interp1d(y_r, x, fill_value='extrapolate')
interpolator['z'] = interp1d(y_z, x, fill_value='extrapolate')

print("To get the same SNR at the current location, required exposure is:")
for camera in dit.SNR:
    print("--- camera {0}: {1}".format(camera, interpolator[camera](np.median(snr[camera][0]))))


Exposure time was set to 10000.0 s in the configuration file
Corresponding SNRs are:
--- camera b: 3.549
--- camera z: 2.050
--- camera r: 2.807
When the telescope is moved, SNR changes to:
--- camera b: 1.132
--- camera z: 0.637
--- camera r: 0.881
To get the same SNR at the current location, required exposure is:
--- camera b: 96206.17242632777
--- camera z: 100422.85853447665
--- camera r: 99402.59674700562


/home/tyapici/Projects/desi/code/specsim/specsim/transform.py:593: UserWarning: Refraction model is inaccurate for altitudes below 5.0 deg.
  .format(low_altitude_threshold))
